# roBERTa paragraph encodings

Encodes paragraphs with roberta for input into snorkel metal end model.

John Brandt


Last Updated: August 15, 2019

In [9]:
import torch
import ssl
import pandas as pd

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

## Encode sentences with roberta

In [11]:
df = pd.read_csv("../data/processed/gold_standard.csv")

In [17]:
def encoder(sentence, max_len = 20):
    sentence_split = sentence.split(' ')
    sentence = ' '.join(sentence_split[:(min(len(sentence_split), max_len))])
    tokens = roberta.encode(sentence)
    tokens = tokens[:min(len(tokens), max_len)]
    last_layer_features = roberta.extract_features(tokens)
    return last_layer_features

range_list = [x for x in range(len(df))]
encodings = [encoder(df['sentences'][x]) for x in range_list[:50]]

In [18]:
encodings_stacked = torch.cat(encodings, 0)
encodings_stacked.shape

torch.Size([50, 20, 1024])

In [19]:
torch.save(encodings_stacked, '../data/interim/encodings.pt')